In [1]:
from binance.client import Client
import configparser
from binance.websockets import BinanceSocketManager
from twisted.internet import reactor

In [2]:
# Loading keys from config file
config = configparser.ConfigParser()
config.read_file(open('/home/venom/GitHub/cheatsheets/medium/algo_trading/binance_API/secret.cfg'))
api_key = config.get('BINANCE', 'API_KEY')
secret_key = config.get('BINANCE', 'SECRET_KEY')

In [3]:
client = Client(api_key, secret_key)

In [4]:
client.API_URL = 'https://testnet.binance.vision/api'

In [5]:
# get balances for all assets & some account information
print(client.get_account())

{'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'canTrade': True, 'canWithdraw': False, 'canDeposit': False, 'updateTime': 1613863611579, 'accountType': 'SPOT', 'balances': [{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}], 'permissions': ['SPOT']}


In [6]:
# get balance for a specific asset only (BTC)
print(client.get_asset_balance(asset='BTC'))

{'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}


In [7]:
# get balances for futures account
# print(client.futures_account_balance())

In [8]:
# get balances for margin account
# print(client.get_margin_account())

In [9]:
# get latest price from Binance API
btc_price = client.get_symbol_ticker(symbol="BTCUSDT")
# print full output (dictionary)
print(btc_price)

{'symbol': 'BTCUSDT', 'price': '7900.00000000'}


In [10]:
print(btc_price["price"])

7900.00000000


In [11]:
btc_price = {'error':False}

def btc_trade_history(msg):
    """ define how to process incoming WebSocket messages """
    print("Inside function")
    if msg['e'] != 'error':
        print(msg['c'])
        btc_price['last'] = msg['c']
        btc_price['bid'] = msg['b']
        btc_price['ask'] = msg['a']
    else:
        btc_price['error'] = True

In [12]:
# init and start the WebSocket
bsm = BinanceSocketManager(client)
conn_key = bsm.start_symbol_ticker_socket('BTCUSDT', btc_trade_history)
bsm.start()

In [13]:
# stop websocket
bsm.stop_socket(conn_key)

# properly terminate WebSocket
reactor.stop()

In [14]:
help(BinanceSocketManager)

Help on class BinanceSocketManager in module binance.websockets:

class BinanceSocketManager(threading.Thread)
 |  BinanceSocketManager(client, user_timeout=1800)
 |  
 |  A class that represents a thread of control.
 |  
 |  This class can be safely subclassed in a limited fashion. There are two ways
 |  to specify the activity: by passing a callable object to the constructor, or
 |  by overriding the run() method in a subclass.
 |  
 |  Method resolution order:
 |      BinanceSocketManager
 |      threading.Thread
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, client, user_timeout=1800)
 |      Initialise the BinanceSocketManager
 |      
 |      :param client: Binance API client
 |      :type client: binance.Client
 |      :param user_timeout: Custom websocket timeout
 |      :type user_timeout: int
 |  
 |  close(self)
 |      Close all connections
 |  
 |  run(self)
 |      Method representing the thread's activity.
 |      
 |      You may override